In [1]:
#기반 클래스 파일 실행
%run adam_model.ipynb

In [5]:
#제한된 기능의 합성곱 계층 및 풀링 계층을 지원할 CnnBasicModel 클래스를 선언
class CnnBasicModel(AdamModel):
    def __init__(self, name, dataset, hconfigs, show_maps = False):
        if isinstance(hconfigs, list) and \
        not isinstance(hconfigs[0], (list, int)):      
            hconfigs = [hconfigs]                      
        self.show_maps = show_maps
        self.need_maps = False
        self.kernels = []
        super(CnnBasicModel, self).__init__(name, dataset, hconfigs)
        self.use_adam = True

In [6]:
#계층에 대한 파라미터 생성 메서드 재정의
def cnn_basic_alloc_layer_param(self, input_shape, hconfig):
    layer_type = get_layer_type(hconfig)             #get_layer_type()함수를 이용해 hconfig에 담긴 계층 이름 A를 layer_type값으로 얻어냄
    
    m_name = 'alloc_{}_layer'.format(layer_type)     #'alloc_A_layer'형식의 메서드 이름을 만들어 m_name에 저장한다.
    method = getattr(self, m_name)                   #이 이름의 메서드를 method 변숫값으로 구해서
    pm, output_shape = method(input_shape, hconfig)  #호출한다.

    return pm, output_shape

CnnBasicModel.alloc_layer_param = cnn_basic_alloc_layer_param

In [7]:
#계층에 대한 순전파 메서드 재정의
def cnn_basic_forward_layer(self, x, hconfig, pm):
    layer_type = get_layer_type(hconfig)
    
    m_name = 'forward_{}_layer'.format(layer_type)
    method = getattr(self, m_name)
    y, aux = method(x, hconfig, pm)            #순전파 처리에 알맞게 인숫값을 전달하고 반환값을 챙기도록 바꿈
        
    return y, aux

CnnBasicModel.forward_layer = cnn_basic_forward_layer

In [8]:
#계층에 대한 역전파 메서드 재정의
def cnn_basic_backprop_layer(self, G_y, hconfig, pm, aux):
    layer_type = get_layer_type(hconfig)
    
    m_name = 'backprop_{}_layer'.format(layer_type)
    method = getattr(self, m_name)
    G_input = method(G_y, hconfig, pm, aux)           #매개변수 구조 및 반환값 내역이 바뀜

    return G_input

CnnBasicModel.backprop_layer = cnn_basic_backprop_layer

In [9]:
#네 가지 계층에 대한 파라미터 생성 메서드 정의
def cnn_basic_alloc_full_layer(self, input_shape, hconfig):
    input_cnt = np.prod(input_shape)
    output_cnt = get_conf_param(hconfig, 'width', hconfig)

    weight = np.random.normal(0, self.rand_std, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

    return {'w':weight, 'b':bias}, [output_cnt]
    
def cnn_basic_alloc_conv_layer(self, input_shape, hconfig):
    assert len(input_shape) == 3                    #input_shape 매개변수로 전달되는 입력 형태가 3차원인지 검사한다.
    xh, xw, xchn = input_shape
    kh, kw = get_conf_param_2d(hconfig, 'ksize')
    ychn = get_conf_param(hconfig, 'chn')

    kernel = np.random.normal(0, self.rand_std, [kh, kw, xchn, ychn])   #커널 크기와 채널 수를 은닉 계층 설정 정보로부터 ksize와 ychn 키값을 이용해 얻어낸다.
    bias = np.zeros([ychn])

    if self.show_maps: self.kernels.append(kernel)  #self.show_maps 플래그가 참인 경우에 한하여 생성된 커널 파라미터를 시각화 정보 수집 리스트인 self.kernels 변수에 추가

    return {'k':kernel, 'b':bias}, [xh, xw, ychn]
    
def cnn_basic_alloc_pool_layer(self, input_shape, hconfig):
    assert len(input_shape) == 3                  #이번 절의 제약 조건, 이미지 크기가 건너뛰기 보폭의 배수가 되는지를 검사한다.
    xh, xw, xchn = input_shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')

    assert xh % sh == 0
    assert xw % sw == 0

    return {}, [xh//sh, xw//sw, xchn]          #파라미터 정보로 빈 사전 구조를 반환한다.

CnnBasicModel.alloc_full_layer = cnn_basic_alloc_full_layer
CnnBasicModel.alloc_conv_layer = cnn_basic_alloc_conv_layer
CnnBasicModel.alloc_max_layer = cnn_basic_alloc_pool_layer
CnnBasicModel.alloc_avg_layer = cnn_basic_alloc_pool_layer

In [10]:
#은닉 계층 구성 정보 접근을 위한 보조 함수 정의
def get_layer_type(hconfig):
    if not isinstance(hconfig, list): return 'full'       #예외 처리에 의해 'full'을 반환한다.
    return hconfig[0]

def get_conf_param(hconfig, key, defval = None):
    if not isinstance(hconfig, list): return defval       #예외 처리에 의해 defval을 반환하는데 이 인숫값은 cnn_basic_alloc_full_layer()함수에서 계층 구성 정보로 8로 주어짐
    if len(hconfig) <= 1: return defval
    if not key in hconfig[1]: return defval
    return hconfig[1][key]
    
def get_conf_param_2d(hconfig, key, defval = None):
    if len(hconfig) <= 1: return defval
    if not key in hconfig[1]: return defval
    val = hconfig[1][key]
    if isinstance(val, list): return val
    return [val, val]

In [11]:
#완전 연결 계층에 대한 순전파 메서드 정의
def cnn_basic_forward_full_layer(self, x, hconfig, pm):
    if pm is None: return x, None
    
    x_org_shape = x.shape
    
    if len(x.shape) != 2:                #입력이 2차원 형태가 아닌 경우
        mb_size = x.shape[0]             #미니배치 크기에 해당하는 첫 번째 차원을 제외한 나머지 차원들을  
        x = x.reshape([mb_size, -1])     #하나의 벡터로 차원 축소하는 처리를 추가
        
    affine = np.matmul(x, pm['w']) + pm['b']
    y = self.activate(affine, hconfig)   #activate()함수를 호출해 이 안에서 은닉 계층 구성 정보에 따라 알맞은 비선형 활성화 함수를 골라 호출하도록 함
    
    return y, [x, y, x_org_shape]

CnnBasicModel.forward_full_layer = cnn_basic_forward_full_layer

In [12]:
#완전 연결 계층에 대한 역전파 메서드 정의
def cnn_basic_backprop_full_layer(self, G_y, hconfig, pm, aux):
    if pm is None: return G_y          #pm값이 None일 때 아무 처리 없이 즉각 반환하는 처리

    x, y, x_org_shape = aux
    
    G_affine = self.activate_derv(G_y, y, hconfig)
    
    g_affine_weight = x.transpose()
    g_affine_input = pm['w'].transpose()
    
    G_weight = np.matmul(g_affine_weight, G_affine)
    G_bias = np.sum(G_affine, axis = 0)
    G_input = np.matmul(G_affine, g_affine_input)
    
    self.update_param(pm, 'w', G_weight)
    self.update_param(pm, 'b', G_bias)

    return G_input.reshape(x_org_shape)       #G_input.reshape()함수를 호출하는 것은 역전파용 보조 정보로 전달된 x_org_shape를 이용해 
                                              #G_input 텐서의 형태를 원래의 입력 형태에 맞추려는 것
CnnBasicModel.backprop_full_layer = cnn_basic_backprop_full_layer

In [13]:
#비선형 활성화 함수에 대한 순전파 및 역전파 메서드 정의
def cnn_basic_activate(self, affine, hconfig):
    if hconfig is None: return affine                #hconfig가 None인 경우 비선형 활성화 함수를 적용하지 않고 바로 반환
    
    func = get_conf_param(hconfig, 'actfunc', 'relu')
    
    if func == 'none':      return affine
    elif func == 'relu':    return relu(affine)
    elif func == 'sigmoid': return sigmoid(affine)
    elif func == 'tanh':    return tanh(affine)
    else:                   assert 0
        
def cnn_basic_activate_derv(self, G_y, y, hconfig):
    if hconfig is None: return G_y                   #hconfig가 None인 경우 비선형 활성화 함수를 적용하지 않고 바로 반환
    
    func = get_conf_param(hconfig, 'actfunc', 'relu')
    
    if func == 'none':      return G_y
    elif func == 'relu':    return relu_derv(y) * G_y
    elif func == 'sigmoid': return sigmoid_derv(y) * G_y
    elif func == 'tanh':    return tanh_derv(y) * G_y
    else:                   assert 0

CnnBasicModel.activate = cnn_basic_activate
CnnBasicModel.activate_derv = cnn_basic_activate_derv

In [14]:
#합성곱 계층에 대한 원론적 순전파 처리 함수 정의
def forward_conv_layer_adhoc(self, x, hconfig, pm):
    mb_size, xh, xw, xchn = x.shape
    kh, kw, _, ychn = pm['k'].shape
    
    conv = np.zeros((mb_size, xh, xw, ychn))
    
    for n in range(mb_size):
        for r in range(xh):
            for c in range(xw):
                for ym in range(ychn):
                    for i in range(kh):
                        for j in range(kw):
                            rx = r + i - (kh-1) // 2
                            cx = c + j - (kw-1) // 2
                            if rx < 0 or rx >= xh: continue   #두 번의 if 명령은 범위를 벗어나는 입력 픽셀을 누적 계산에서 배제시키기 위한 것으로
                            if cx < 0 or cx >= xw: continue   #이 처리 덕분에 범위 바깥의 입력값은 0으로 간주하는 효과를 갖게 된다.
                            for xm in range(xchn):
                                kval = pm['k'][i][j][xm][ym]
                                ival = x[n][rx][cx][xm]
                                conv[n][r][c][ym] += kval * ival

    y = self.activate(conv + pm['b'], hconfig)    #편향은 커널 가중치 처리 결과 얻어진 4차원 텐서에 편향 벡터를 일괄적으로 더하는 방법으로 반영한다.
    
    return y, [x, y]

In [15]:
#합성곱 계층에 대한 개선된 순전파 처리 함수 정의
def forward_conv_layer_better(self, x, hconfig, pm):
    mb_size, xh, xw, xchn = x.shape
    kh, kw, _, ychn = pm['k'].shape
    
    conv = np.zeros((mb_size, xh, xw, ychn))

    bh, bw = (kh-1)//2, (kw-1)//2
    eh, ew = xh + kh - 1, xw + kw - 1
    
    x_ext = np.zeros((mb_size, eh, ew, xchn))   #반복 처리 전에 커널 크기를 고려해 확장된 x_ext버퍼를 준비해 0으로 채운후
    x_ext[:, bh:bh + xh, bw:bw + xw, :] = x    #버퍼의 중앙 부분에 입력을 복사
    
    k_flat = pm['k'].transpose([3, 0, 1, 2]).reshape([ychn, -1])   #출력 채널별로 커널의 나머지 세 차원을 한 차원 벡터로 축소시켜 k_flat에 저장
    
    for n in range(mb_size):
        for r in range(xh):
            for c in range(xw):
                for ym in range(ychn):
                    xe_flat = x_ext[n, r:r + kh, c:c + kw, :].flatten()  #확장된 x_ext 버퍼에서 출력 픽셀 위치에 대응하는 커널 면적의 사각 영역을 지정해 차원을 축소하여 내적 계산에 이용할 벡터 xe_flat을 구한다.
                    conv[n, r, c, ym] = (xe_flat*k_flat[ym]).sum()    #xe_flat벡터와 k_flat[ym]벡터의 내적을 계산하는 방법으로 간단히 출력 픽셀 하나의 값을 구할 수 있다.
                    
    y = self.activate(conv + pm['b'], hconfig)
    
    return y, [x, y]

In [16]:
#합성곱 계층에 대한 순전파 메서드 정의
def cnn_basic_forward_conv_layer(self, x, hconfig, pm):
    mb_size, xh, xw, xchn = x.shape
    kh, kw, _, ychn = pm['k'].shape
    
    x_flat = get_ext_regions_for_conv(x, kh, kw)
    k_flat = pm['k'].reshape([kh*kw*xchn, ychn])     #4차원 텐서 pm['k']를 2차원 행렬 k_flat으로 바꿈
    conv_flat = np.matmul(x_flat, k_flat)
    conv = conv_flat.reshape([mb_size, xh, xw, ychn]) #[mb_size, xh, xw, ychn]형태의 4차원 형태로 재해석

    y = self.activate(conv + pm['b'], hconfig)      #출력 채널마다 하나씩 값을 갖는 편향 벡터값을 더하고 비선형 활성화 함수 적용

    if self.need_maps: self.maps.append(y)      #생성된 출력 y를 시각화 정보를 수집하는 self.maps 리스트에 추가한다.
    
    return y, [x_flat, k_flat, x, y]

CnnBasicModel.forward_conv_layer = cnn_basic_forward_conv_layer

In [17]:
#합성곱 계층에 대한 역전파 메서드 정의
def cnn_basic_backprop_conv_layer(self, G_y, hconfig, pm, aux):
    x_flat, k_flat, x, y = aux
    
    kh, kw, xchn, ychn = pm['k'].shape
    mb_size, xh, xw, _ = G_y.shape
    
    G_conv = self.activate_derv(G_y, y, hconfig)   #비선형 활성화 함수에 대한 역전파 처리를 수행

    G_conv_flat = G_conv.reshape(mb_size*xh*xw, ychn)     #2차원 행렬 conv_flat을 4차원 텐서 conv로 재해석했던 순전파 과정에서 처리에
                                                          #대한 역처리로서 4차원 텐서인 G_conv를 conv_flat과 같은 형태의 2차원 행렬 G_conv_flat으로 재해석
    g_conv_k_flat = x_flat.transpose()
    g_conv_x_flat = k_flat.transpose()
    
    G_k_flat = np.matmul(g_conv_k_flat, G_conv_flat)
    G_x_flat = np.matmul(G_conv_flat, g_conv_x_flat)
    G_bias = np.sum(G_conv_flat, axis = 0)
    
    G_kernel = G_k_flat.reshape([kh, kw, xchn, ychn])     #2차원의 G_K_flat을 커널의 원래 형태인 4차원 텐서로 재해석하여 G_kernel을 얻음
    G_input = undo_ext_regions_for_conv(G_x_flat, x, kh, kw)
    
    self.update_param(pm, 'k', G_kernel)
    self.update_param(pm, 'b', G_bias)
    
    return G_input

CnnBasicModel.backprop_conv_layer = cnn_basic_backprop_conv_layer

In [18]:
#확장 영역 처리 함수 정의
def get_ext_regions_for_conv(x, kh, kw):
    mb_size, xh, xw, xchn = x.shape

    regs = get_ext_regions(x, kh, kw, 0)
    regs = regs.transpose([2, 0, 1, 3, 4, 5])
    
    return regs.reshape([mb_size*xh*xw, kh*kw*xchn])

def get_ext_regions(x, kh, kw, fill):
    mb_size, xh, xw, xchn = x.shape
    
    eh, ew = xh + kh - 1, xw + kw - 1
    bh, bw = (kh-1)//2, (kw-1)//2

    x_ext = np.zeros((mb_size, eh, ew, xchn), dtype = 'float32') + fill
    x_ext[:, bh:bh + xh, bw:bw + xw, :] = x
    
    regs = np.zeros((xh, xw, mb_size*kh*kw*xchn), dtype = 'float32')

    for r in range(xh):
        for c in range(xw):
            regs[r, c, :] = x_ext[:, r:r + kh, c:c + kw, :].flatten()

    return regs.reshape([xh, xw, mb_size, kh, kw, xchn])

In [19]:
#확장 영역에 대한 역처리 함수 정의
def undo_ext_regions_for_conv(regs, x, kh, kw):
    mb_size, xh, xw, xchn = x.shape

    regs = regs.reshape([mb_size, xh, xw, kh, kw, xchn])    #순전파와 반대순서로 반대 작업하기위해 regs텐서의 형태를 재해석
    regs = regs.transpose([1, 2, 0, 3, 4, 5])               #맨 앞의 미니배치 축을 세 번째 축으로 옮긴다.
    
    return undo_ext_regions(regs, kh, kw)     #함수에 대한 역처리를 수행하는 undo_ext_regions()함수 호출

def undo_ext_regions(regs, kh, kw):
    xh, xw, mb_size, kh, kw, xchn = regs.shape
    
    eh, ew = xh + kh - 1, xw + kw - 1
    bh, bw = (kh-1)//2, (kw-1)//2

    gx_ext = np.zeros([mb_size, eh, ew, xchn], dtype = 'float32')

    for r in range(xh):
        for c in range(xw):
            gx_ext[:, r:r + kh, c:c + kw, :] += regs[r, c]

    return gx_ext[:, bh:bh + xh, bw:bw + xw, :]

In [20]:
#평균치 풀링 계층에 대한 순전파와 역전파 메서드 정의
def cnn_basic_forward_avg_layer(self, x, hconfig, pm):
    mb_size, xh, xw, chn = x.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    yh, yw = xh // sh, xw // sw

    x1 = x.reshape([mb_size, yh, sh, yw, sw, chn])
    x2 = x1.transpose(0, 1, 3, 5, 2, 4)
    x3 = x2.reshape([-1, sh*sw])
    
    y_flat = np.average(x3, 1)
    y = y_flat.reshape([mb_size, yh, yw, chn])
    
    if self.need_maps: self.maps.append(y)

    return y, None

def cnn_basic_backprop_avg_layer(self, G_y, hconfig, pm, aux):
    mb_size, yh, yw, chn = G_y.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    xh, xw = yh * sh, yw * sw
    
    gy_flat = G_y.flatten() / (sh * sw)

    gx1 = np.zeros([mb_size*yh*yw*chn, sh*sw], dtype = 'float32')
    for i in range(sh*sw):
        gx1[:, i] = gy_flat
    gx2 = gx1.reshape([mb_size, yh, yw, chn, sh, sw])
    gx3 = gx2.transpose([0, 1, 4, 2, 5, 3])

    G_input = gx3.reshape([mb_size, xh, xw, chn])
        
    return G_input

CnnBasicModel.forward_avg_layer = cnn_basic_forward_avg_layer
CnnBasicModel.backprop_avg_layer = cnn_basic_backprop_avg_layer

In [21]:
#최대치 풀링 계층에 대한 순전파와 역전파 메서드 정의
def cnn_basic_forward_max_layer(self, x, hconfig, pm):
    mb_size, xh, xw, chn = x.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    yh, yw = xh // sh, xw // sw

    x1 = x.reshape([mb_size, yh, sh, yw, sw, chn])
    x2 = x1.transpose(0, 1, 3, 5, 2, 4)
    x3 = x2.reshape([-1, sh*sw])

    idxs = np.argmax(x3, axis = 1)       #np.argmax()함수를 이용해 최대치 위치를 idxs 벡터에 모은다.
    y_flat = x3[np.arange(mb_size*yh*yw*chn), idxs]    #이를 인덱스 삼아 최대치 벡터 y_flat을 구하는 방식으로 처리한다.(idxs는 역전파용 보조 정보)
    y = y_flat.reshape([mb_size, yh, yw, chn])
    
    if self.need_maps: self.maps.append(y)

    return y, idxs

def cnn_basic_backprop_max_layer(self, G_y, hconfig, pm, aux):
    idxs = aux
    
    mb_size, yh, yw, chn = G_y.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    xh, xw = yh * sh, yw * sw
    
    gy_flat = G_y.flatten()

    gx1 = np.zeros([mb_size*yh*yw*chn, sh*sw], dtype = 'float32')
    gx1[np.arange(mb_size*yh*yw*chn), idxs] = gy_flat[:]
    gx2 = gx1.reshape([mb_size, yh, yw, chn, sh, sw])
    gx3 = gx2.transpose([0, 1, 4, 2, 5, 3])

    G_input = gx3.reshape([mb_size, xh, xw, chn])
        
    return G_input

CnnBasicModel.forward_max_layer = cnn_basic_forward_max_layer
CnnBasicModel.backprop_max_layer = cnn_basic_backprop_max_layer

In [22]:
#시각화 메서드 정의
def cnn_basic_visualize(self, num):
    print('Model {} Visualization'.format(self.name))
    
    self.need_maps = self.show_maps    #특징맵 수집 여부를 지정하며 특징맵 수집 버퍼인 self.maps 리스트도 공백으로 초기화
    self.maps = []

    deX, deY = self.dataset.get_visualize_data(num)
    est = self.get_estimate(deX)

    if self.show_maps:
        for kernel in self.kernels:                  #커널 출력을 수행하는데 각 합성곱 계층의 커널에 대해 이미지 해상도와 관련 없는
            kh, kw, xchn, ychn = kernel.shape        #두 축을 앞으로 옮긴 후 합병하고 나서 앞의 다섯 개를 고르는 방법으로 이미지 해상도와 관련된 두 축을 갖는 2차원 형태의 커널 슬라이스를 추출해 출력한다.
            grids = kernel.reshape([kh, kw, -1]).transpose(2, 0, 1)
            draw_images_horz(grids[0:5, :, :])

        for pmap in self.maps:                  #합성곱 계층이나 풀링 계층의 출력 모두를 수집한 self.maps 리스트 내용을 차례로 출력하되
            draw_images_horz(pmap[:, :, :, 0])  #출력 대상을 첫 번째 채널로 제한해 지나치게 많은 출력을 막았다.
        
    self.dataset.visualize(deX, est, deY)       #데이터셋 객체에 따른 시각화 내용 출력

    self.need_maps = False       #self.need_maps 플래그와 self.maps 버퍼를 청소하여 향후 실행에 잘못된 영향이나 메모리 부담을 미치지 않도록 한다.
    self.maps = None

CnnBasicModel.visualize = cnn_basic_visualize